In [0]:
# prerequisites for imgaug library
!pip install six numpy scipy Pillow matplotlib scikit-image opencv-python imageio Shapely

In [0]:
# image augmentation library imgaug (https://github.com/aleju/imgaug)
!pip install imgaug

# NOTE: make sure to Restart Runtime after this installation completes

    100% |████████████████████████████████| 17.3MB 2.5MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


In [0]:
# imgaug needs latest scikit module. so need to do this. else, will get some weird error related to numpy!
!pip install --upgrade scikit-image

# NOTE: make sure to Restart Runtime after this installation completes

    100% |████████████████████████████████| 26.3MB 1.6MB/s 
    100% |████████████████████████████████| 2.0MB 12.1MB/s 
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 4.1.1
    Uninstalling Pillow-4.1.1:
      Successfully uninstalled Pillow-4.1.1
  Found existing installation: scikit-image 0.13.1
    Uninstalling scikit-image-0.13.1:
      Successfully uninstalled scikit-image-0.13.1


In [0]:
from imgaug import augmenters as iaa

In [0]:
seq = iaa.Sequential([iaa.Sometimes(0.5, iaa.SomeOf(3,[iaa.Fliplr(1.0),
                                                    iaa.CoarseDropout(0.02, size_percent=0.05),
                                                    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),
                                                    iaa.GaussianBlur(sigma=(0.0, 1.0)),
                                                    iaa.Multiply((0.5, 1.5)),
                                                    iaa.Affine(rotate=(-30, 30)),
                                                    iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)),
                                                    iaa.Affine(scale=(0.5, 1.5))]))])

***Eight Augmentations used***

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib  inline
from keras.preprocessing.image import ImageDataGenerator

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as K

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls
!unzip -qq 'gdrive/My Drive/Colab Notebooks/Assg4/tiny-imagenet-200.zip'
!ls

gdrive	sample_data
gdrive	sample_data  tiny-imagenet-200


In [0]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

train_generator = ImageDataGenerator(
    rescale= 1./255,
    preprocessing_function=seq.augment_image
    )

valid_generator = ImageDataGenerator(rescale=1./255)



In [0]:
train_generator = train_generator.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=64, class_mode='categorical', shuffle=True, seed=42)



Found 100000 images belonging to 200 classes.


In [0]:
validation_generator = valid_generator.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=512, shuffle=True, seed=42)

Found 10000 images belonging to 200 classes.


In [0]:
from __future__ import division
import six
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    Dropout
)
from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D,
    SeparableConv2D
)
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
#from keras import backend as K

In [0]:
def _bn_relu(input):
    """Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)
  
  
def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = SeparableConv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f
  
  
def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return SeparableConv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f

In [0]:
def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    #print(input_shape, residual_shape )
    #stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    #stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]
   
    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if not equal_channels:
        shortcut = SeparableConv2D(filters=residual_shape[CHANNEL_AXIS],  #1x1 used to increase channels, as same number of channels are required for adding two layers
                          kernel_size=(1, 1),
                          strides=(1,1),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])

In [0]:
def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks.
    """
    def f(input):
        for i in range(repetitions):
            #print("r",i)
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (1, 1)
            input = block_function(filters=filters, init_strides=init_strides,
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input

    return f
  
  
def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = SeparableConv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        #conv1 = Dropout(0.2)(conv1)
        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        #print("basic ", K.int_shape(input), K.int_shape(residual))
        #return residual
        return _shortcut(input, residual)

    return f

In [0]:
def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3
        
        
def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier

In [0]:
class Builder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras `Model`.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_dim_ordering() == 'tf':
            input_shape = (input_shape[1], input_shape[2], input_shape[0])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)
        
        filters = 32

        input = Input(shape=input_shape)
        conv1 = Conv2D(filters, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False, kernel_regularizer = l2(1.e-4))(input)
        #conv1 = _conv_bn_relu(filters=filters, kernel_size=(3, 3), strides=(2, 2))(input)
        conv1_br = _bn_relu(conv1)
        conv2 = _conv_bn_relu(filters=filters, kernel_size=(3, 3), strides=(1, 1))(conv1_br)
        conv3 = _conv_bn_relu(filters=filters, kernel_size=(3, 3), strides=(1, 1))(conv2)
        #pool1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")(conv3)
        #pool1 = Dropout(0.2)(conv3)

        block = conv3
        filters = 128
        for i, r in enumerate(repetitions):
            #print(i, K.int_shape(block), K.int_shape(input), filters)
            #print("i",i)
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            if i != 2:
              block = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")(block)
              filters *= 4
            #print(i, K.int_shape(block), K.int_shape(input), filters)
           # print(filters)

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        block = Conv2D(num_outputs, (1,1), strides=(1,1), padding='same', name='before_flatten', use_bias=False)(block)
        pool2 = GlobalAveragePooling2D()(block)
        #flatten1 = Flatten()(pool2)
        #dense = Dense(units=num_outputs, kernel_initializer="he_normal",
                      #activation="softmax")(flatten1)

        out = Activation('softmax')(pool2)
        model = Model(inputs=input, outputs=out)
        return model

    @staticmethod
    def build_model(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2])


In [0]:
model = Builder.build_model((3, None, None), 200)
#print(filters)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv_1[0][0]                     
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
separable_

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
from keras.callbacks import *
filepath = "/content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m.hdf5"
checkpoint = [ModelCheckpoint(filepath, period = 1, monitor='val_acc', verbose=1,save_weights_only=True, save_best_only=True)]

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=checkpoint,
                    epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
390/390 [==============================] - 745s 2s/step - loss: 4.0411 - acc: 0.1410 - val_loss: 6.2002 - val_acc: 0.0627

Epoch 00001: val_acc improved from -inf to 0.06271, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m.hdf5
Epoch 2/5
390/390 [==============================] - 729s 2s/step - loss: 3.1260 - acc: 0.2865 - val_loss: 4.6728 - val_acc: 0.1105

Epoch 00002: val_acc improved from 0.06271 to 0.11046, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m.hdf5
Epoch 3/5
390/390 [==============================] - 721s 2s/step - loss: 2.6258 - acc: 0.3783 - val_loss: 4.3594 - val_acc: 0.1203

Epoch 00003: val_acc improved from 0.11046 to 0.12026, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m.hdf5
Epoch 4/5
390/390 [==============================] - 719s 2s/step - loss: 2.2543 - acc: 0.4496 - val_loss: 4.0648 - val_acc: 0.1656

Epoch 00004: va

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=checkpoint,
                    epochs=5)

Epoch 1/5
390/390 [==============================] - 275s 706ms/step - loss: 3.1496 - acc: 0.2866 - val_loss: 5.7871 - val_acc: 0.0518

Epoch 00001: val_acc did not improve from 0.16558
Epoch 2/5
390/390 [==============================] - 271s 695ms/step - loss: 2.5025 - acc: 0.4000 - val_loss: 5.6149 - val_acc: 0.0522

Epoch 00002: val_acc did not improve from 0.16558
Epoch 3/5
390/390 [==============================] - 270s 692ms/step - loss: 1.9873 - acc: 0.5022 - val_loss: 6.2201 - val_acc: 0.0417

Epoch 00003: val_acc did not improve from 0.16558
Epoch 4/5
390/390 [==============================] - 270s 692ms/step - loss: 1.4364 - acc: 0.6257 - val_loss: 7.9510 - val_acc: 0.0296

Epoch 00004: val_acc did not improve from 0.16558
Epoch 5/5
390/390 [==============================] - 270s 691ms/step - loss: 0.8262 - acc: 0.7774 - val_loss: 7.7326 - val_acc: 0.0413

Epoch 00005: val_acc did not improve from 0.16558


In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=checkpoint,
                    epochs=10)

Epoch 1/10
1562/1562 [==============================] - 3037s 2s/step - loss: 2.4521 - acc: 0.4166 - val_loss: 2.6662 - val_acc: 0.3917

Epoch 00001: val_acc improved from 0.16558 to 0.39165, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m.hdf5
Epoch 2/10
1562/1562 [==============================] - 3044s 2s/step - loss: 1.7677 - acc: 0.5562 - val_loss: 2.4124 - val_acc: 0.4317

Epoch 00002: val_acc improved from 0.39165 to 0.43170, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m.hdf5
Epoch 3/10
1562/1562 [==============================] - 3123s 2s/step - loss: 1.3799 - acc: 0.6418 - val_loss: 2.2364 - val_acc: 0.4816

Epoch 00003: val_acc improved from 0.43170 to 0.48156, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m.hdf5
Epoch 4/10
1562/1562 [==============================] - 3083s 2s/step - loss: 0.9964 - acc: 0.7310 - val_loss: 2.3172 - val_acc: 0.4859

Epoch 00004: val_acc improved from 0.48156 t

In [0]:
from keras.callbacks import *
filepath = "/content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m_aug.hdf5"
checkpoint = [ModelCheckpoint(filepath, period = 1, monitor='val_acc', verbose=1,save_weights_only=True, save_best_only=True)]

In [0]:
#from keras.models import load_weights
model.load_weights("/content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m.hdf5", by_name=True)

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=checkpoint,
                    epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
390/390 [==============================] - 760s 2s/step - loss: 2.0992 - acc: 0.4909 - val_loss: 2.9954 - val_acc: 0.3293

Epoch 00001: val_acc improved from -inf to 0.32926, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m_aug.hdf5
Epoch 2/5
390/390 [==============================] - 743s 2s/step - loss: 1.5529 - acc: 0.6048 - val_loss: 3.3185 - val_acc: 0.2941

Epoch 00002: val_acc did not improve from 0.32926
Epoch 3/5
390/390 [==============================] - 735s 2s/step - loss: 1.2286 - acc: 0.6797 - val_loss: 3.2307 - val_acc: 0.3231

Epoch 00003: val_acc did not improve from 0.32926
Epoch 4/5
390/390 [==============================] - 735s 2s/step - loss: 0.9880 - acc: 0.7413 - val_loss: 3.2592 - val_acc: 0.3142

Epoch 00004: val_acc did not improve from 0.32926
Epoch 5/5
390/390 [==============================] - 735s 2s/step - loss: 0.8084 - acc: 0.7878 - val_loss: 3.4357 - val_acc: 0.3119



In [0]:
from keras.callbacks import *
filepath = "/content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m_aug.hdf5"
checkpoint = ModelCheckpoint(filepath, period = 1, monitor='val_acc', verbose=1,save_weights_only=True, save_best_only=True)

In [0]:
from keras.callbacks import ReduceLROnPlateau

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=2, min_lr=0.001)

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[checkpoint,reduce_lr],
                    epochs=5)

Epoch 1/5
390/390 [==============================] - 353s 904ms/step - loss: 2.6294 - acc: 0.3776 - val_loss: 5.4215 - val_acc: 0.0851

Epoch 00001: val_acc improved from -inf to 0.08512, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m_aug.hdf5
Epoch 2/5
390/390 [==============================] - 339s 869ms/step - loss: 1.9449 - acc: 0.5114 - val_loss: 5.8040 - val_acc: 0.0809

Epoch 00002: val_acc did not improve from 0.08512
Epoch 3/5
390/390 [==============================] - 331s 848ms/step - loss: 1.5439 - acc: 0.6034 - val_loss: 6.5448 - val_acc: 0.0716

Epoch 00003: val_acc did not improve from 0.08512
Epoch 4/5
390/390 [==============================] - 338s 866ms/step - loss: 1.2205 - acc: 0.6853 - val_loss: 6.9621 - val_acc: 0.0772

Epoch 00004: val_acc did not improve from 0.08512
Epoch 5/5
390/390 [==============================] - 324s 830ms/step - loss: 0.9816 - acc: 0.7461 - val_loss: 6.7158 - val_acc: 0.0839

Epoch 00005: val_acc did not impr

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[checkpoint,reduce_lr],
                    epochs=10)

Epoch 1/10
1562/1562 [==============================] - 3128s 2s/step - loss: 1.5639 - acc: 0.5975 - val_loss: 2.2390 - val_acc: 0.5026

Epoch 00001: val_acc improved from 0.08512 to 0.50257, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m_aug.hdf5
Epoch 2/10
1562/1562 [==============================] - 3147s 2s/step - loss: 0.9879 - acc: 0.7365 - val_loss: 2.3136 - val_acc: 0.5091

Epoch 00002: val_acc improved from 0.50257 to 0.50906, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m_aug.hdf5
Epoch 3/10
1562/1562 [==============================] - 3140s 2s/step - loss: 0.7730 - acc: 0.7890 - val_loss: 2.3389 - val_acc: 0.5218

Epoch 00003: val_acc improved from 0.50906 to 0.52182, saving model to /content/gdrive/My Drive/Colab Notebooks/Assg4/assg4_11_16m_aug.hdf5
Epoch 4/10
1562/1562 [==============================] - 3124s 2s/step - loss: 0.6228 - acc: 0.8279 - val_loss: 2.5758 - val_acc: 0.5052

Epoch 00004: val_acc did not imp

KeyboardInterrupt: ignored

***Best accuracy achieved(after applying augmentaion and ReduceLROnPlateau) =  52.182***

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git 
import keras_contrib
from keras_contrib.callbacks.cyclical_learning_rate import CyclicLR

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-8u6vrlac
  Stored in directory: /tmp/pip-ephem-wheel-cache-v5w5k9_v/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
cyclic = CyclicLR(base_lr=1e-4, max_lr=0.05, step_size=4000, mode='triangular')
model.fit_generator(train_generator, epochs=10, steps_per_epoch=800, validation_steps=40, validation_data=validation_generator, callbacks = [checkpoint, cyclic]) 

Epoch 1/10
800/800 [==============================] - 1758s 2s/step - loss: 1.9684 - acc: 0.5236 - val_loss: 4.6034 - val_acc: 0.1755

Epoch 00001: val_acc did not improve from 0.52182
Epoch 2/10
800/800 [==============================] - 1756s 2s/step - loss: 2.6624 - acc: 0.3748 - val_loss: 4.0501 - val_acc: 0.2283

Epoch 00002: val_acc did not improve from 0.52182
Epoch 3/10
800/800 [==============================] - 1759s 2s/step - loss: 2.8150 - acc: 0.3505 - val_loss: 5.5032 - val_acc: 0.1284

Epoch 00003: val_acc did not improve from 0.52182
Epoch 4/10
292/800 [=========>....................] - ETA: 16:36 - loss: 2.9157 - acc: 0.3333

***Applied cycliclr***


***Best accuracy achieved(after applying augmentaion and ReduceLROnPlateau) = 52.182, Eight Augmentations used***